In [ ]:
import rasterio
from rio_tiler.io import Reader
from vsifile.rasterio import opener

import logging

logging.basicConfig(level=logging.DEBUG)

# turn off some logs
logging.getLogger("rasterio").setLevel(logging.ERROR)
logging.getLogger("httpcore").setLevel(logging.ERROR)
logging.getLogger("botocore").setLevel(logging.ERROR)
logging.getLogger("httpx").setLevel(logging.ERROR)


src_path = "https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/15/T/VK/2023/10/S2B_15TVK_20231008_0_L2A/TCI.tif"
z, x, y = 11, 493, 741


In [ ]:
import attr
from rio_tiler.io import Reader

@attr.s
class VSIReader(Reader):

    def __attrs_post_init__(self):
        if not self.dataset:
            self.dataset = self._ctx_stack.enter_context(rasterio.open(self.input, opener=opener))
        super().__attrs_post_init__()

In [ ]:
%%timeit -n 1 -r 1

with rasterio.Env(
    GDAL_DISABLE_READDIR_ON_OPEN="EMPTY_DIR",
    GDAL_INGESTED_BYTES_AT_OPEN=32768,
    GDAL_HTTP_MERGE_CONSECUTIVE_RANGES="YES",
    CPL_DEBUG="ON",
    CPL_CURL_VERBOSE="YES",
    GDAL_HTTP_MULTIRANGE="YES",
    GDAL_HTTP_VERSION="2",
):
    with VSIReader(src_path) as src:
        src.preview()

In [ ]:
%%timeit -n 1 -r 1

with rasterio.Env(
    GDAL_DISABLE_READDIR_ON_OPEN="EMPTY_DIR",
    GDAL_INGESTED_BYTES_AT_OPEN=32768,
    GDAL_HTTP_MERGE_CONSECUTIVE_RANGES="YES",
    CPL_DEBUG="ON",
    CPL_CURL_VERBOSE="YES",
    GDAL_HTTP_MULTIRANGE="YES",
    GDAL_HTTP_VERSION="2",
):
    with VSIReader(src_path) as src:
        src.tile(x, y, z)